# Analyzing SD vendor checkbook data

A starting point for loading [the state's vendor checkbook CSV files](https://open.sd.gov/vendor.aspx) and analyzing the data in pandas.

In [1]:
from datetime import datetime

import pandas as pd
from fetch_latest_data import csv_read_settings

In [2]:
# no scientific notation for big numbers
pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [3]:
# uncomment and run to refresh local csv
# %run fetch_latest_data

In [4]:
df = pd.read_csv('sd-vendor-checkbook.csv', **csv_read_settings)

In [5]:
df.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt
0,010,2022-07-05,01307002,GRANT COUNTY REVIEW INC,12403860,NaN,2022-07-08,001134,48.00
1,010,2022-07-05,111,BLACK HILLS LODGING LLC,12620107,NaN,2022-07-08,001138,75.00
2,010,2022-07-05,48646,SD HERITAGE STORE,STATE,83,2022-07-08,001131,308.40
3,010,2022-07-05,IN3835168,INNOVATIVE OFFICE SOLUTIONS,12550348,NaN,2022-07-08,001130,15.07
4,010,2022-07-05,IN3843549,INNOVATIVE OFFICE SOLUTIONS,12550348,NaN,2022-07-08,001142,26.24


### Join to agency name data

In [6]:
# uncomment and run to get a fresh copy of agency codes table
# %run fetch_agency_codes

In [7]:
df_codes = pd.read_csv('sd-agency-codes.csv', dtype={'agency_code': str})

In [8]:
df_codes.head()

,agency_code,agency_name
0,010,GOVERNOR'S OFFICE
1,011,BUREAU OF FINANCE & MANAGEMENT
2,012,BUREAU OF ADMINISTRATION
3,013,BUREAU OF INFORMATION & TELE.
4,014,BUREAU OF HUMAN RESOURCES


In [9]:
# merge the two
df_merged = pd.merge(df,
                  df_codes,
                  how='left',
                  left_on='agency',
                  right_on='agency_code')

In [10]:
df_merged.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name
0,010,2022-07-05,01307002,GRANT COUNTY REVIEW INC,12403860,NaN,2022-07-08,001134,48.00,010,GOVERNOR'S OFFICE
1,010,2022-07-05,111,BLACK HILLS LODGING LLC,12620107,NaN,2022-07-08,001138,75.00,010,GOVERNOR'S OFFICE
2,010,2022-07-05,48646,SD HERITAGE STORE,STATE,83,2022-07-08,001131,308.40,010,GOVERNOR'S OFFICE
3,010,2022-07-05,IN3835168,INNOVATIVE OFFICE SOLUTIONS,12550348,NaN,2022-07-08,001130,15.07,010,GOVERNOR'S OFFICE
4,010,2022-07-05,IN3843549,INNOVATIVE OFFICE SOLUTIONS,12550348,NaN,2022-07-08,001142,26.24,010,GOVERNOR'S OFFICE


In [11]:
# find agency codes with no match --
# ran this once and then checked w/ agency
# on missing codes in lookup table
'''
no_match = df_merged[df_merged['agency_code'].isnull()]
no_match['agency'].unique()
no_match.to_clipboard(index=False)
'''

"\nno_match = df_merged[df_merged['agency_code'].isnull()]\nno_match['agency'].unique()\nno_match.to_clipboard(index=False)\n"

In [59]:
# make sure we ended up with the same number of records
assert(len(df) == len(df_merged))

### Explore payments to a specific vendor

As an example, let's check out payments to a couple of cities in the Northern Hills.

In [13]:
# show me records where the vendor name contains any of these city names
df_merged[df_merged['vendor_name'].str.contains('spearfish|lead|deadwood|whitewood', case=False)]

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name
10,010,2022-07-11,1919-110-02,CITY OF WHITEWOOD,12055252,NaN,2022-07-15,007358,72663.91,010,GOVERNOR'S OFFICE
180,010,2022-08-29,1919-110-03,CITY OF WHITEWOOD,12055252,NaN,2022-09-14,046613,54764.34,010,GOVERNOR'S OFFICE
496,010,2022-09-27,1919-110-04,CITY OF WHITEWOOD,12055252,NaN,2022-10-05,066605,47279.58,010,GOVERNOR'S OFFICE
565,010,2022-10-25,1919-110-05,CITY OF WHITEWOOD,12055252,NaN,2022-11-09,088570,169879.55,010,GOVERNOR'S OFFICE
759,011,2022-08-31,M01CLFRFT2BASE,CITY OF LEAD,12054774,CV,2022-09-09,M051545,263143.38,011,BUREAU OF FINANCE & MANAGEMENT
...,...,...,...,...,...,...,...,...,...,...,...
618117,29,2021-04-28,042821,DEADWOOD RESORT LLC,12158069,NaN,2021-06-25,671551,1513.94,29,ATTORNEY GENERAL
618421,27,2021-06-03,BOX 494 - 2021,US POSTAL SERVICE-DEADWOOD,12005421,46,2021-06-18,664993,92.00,27,UNIFIED JUDICIAL SYSTEMS
618595,30,2021-02-10,M3021CS2021,SPEARFISH SCHOOL DISTRICT 40-2,12031379,01,2021-02-10,M576777,202168.85,30,SCHOOL & PUBLIC LANDS
619351,29,2020-12-23,12/23/20 AWARD,CITY OF LEAD,12054774,NaN,2021-01-06,550602,28602.00,29,ATTORNEY GENERAL


In [14]:
# drop the vendor numbers of interest into a list
vendor_ids = [
    '12054774',
    '12054391',
    '12055082',
    '12055252'
]

In [15]:
# filter into a new df
df_northern_hills = df_merged[df_merged['vendor_number'].isin(vendor_ids)]

In [16]:
df_northern_hills.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name
10,010,2022-07-11,1919-110-02,CITY OF WHITEWOOD,12055252,NaN,2022-07-15,007358,72663.91,010,GOVERNOR'S OFFICE
180,010,2022-08-29,1919-110-03,CITY OF WHITEWOOD,12055252,NaN,2022-09-14,046613,54764.34,010,GOVERNOR'S OFFICE
496,010,2022-09-27,1919-110-04,CITY OF WHITEWOOD,12055252,NaN,2022-10-05,066605,47279.58,010,GOVERNOR'S OFFICE
565,010,2022-10-25,1919-110-05,CITY OF WHITEWOOD,12055252,NaN,2022-11-09,088570,169879.55,010,GOVERNOR'S OFFICE
759,011,2022-08-31,M01CLFRFT2BASE,CITY OF LEAD,12054774,CV,2022-09-09,M051545,263143.38,011,BUREAU OF FINANCE & MANAGEMENT


### Break down spending by month

Add a `yearmonth` column to allow grouping by month.

In [17]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
df_merged['yearmonth'] = df_merged['ap_payment_date'].dt.strftime('%Y%m')

In [18]:
df_merged.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name,yearmonth
0,010,2022-07-05,01307002,GRANT COUNTY REVIEW INC,12403860,NaN,2022-07-08,001134,48.00,010,GOVERNOR'S OFFICE,202207
1,010,2022-07-05,111,BLACK HILLS LODGING LLC,12620107,NaN,2022-07-08,001138,75.00,010,GOVERNOR'S OFFICE,202207
2,010,2022-07-05,48646,SD HERITAGE STORE,STATE,83,2022-07-08,001131,308.40,010,GOVERNOR'S OFFICE,202207
3,010,2022-07-05,IN3835168,INNOVATIVE OFFICE SOLUTIONS,12550348,NaN,2022-07-08,001130,15.07,010,GOVERNOR'S OFFICE,202207
4,010,2022-07-05,IN3843549,INNOVATIVE OFFICE SOLUTIONS,12550348,NaN,2022-07-08,001142,26.24,010,GOVERNOR'S OFFICE,202207


In [19]:
# pivot table to show spending by vendor by month
pivot_by_vendor_by_month = pd.pivot_table(df_merged,
                              index='vendor_number',
                              columns='yearmonth',
                              values='amt',
                              aggfunc=sum).reset_index().fillna(0.0)

In [20]:
pivot_by_vendor_by_month.head()

yearmonth,vendor_number,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211
0,12001820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9218.52,1390.18
1,12001823,53353.87,52533.01,52723.84,52346.98,52186.30,52215.97,51974.41,52402.83,51845.38,...,51501.84,52557.13,53015.49,52589.16,51873.09,51578.06,50749.81,51047.36,51306.75,52057.66
2,12001827,0.00,0.00,0.00,0.00,0.00,0.00,675.00,0.00,736.00,...,0.00,0.00,0.00,0.00,372.00,0.00,0.00,0.00,0.00,0.00
3,12001831,3029.28,10061.58,11085.74,4028.23,0.00,8776.72,6357.01,3394.56,5616.32,...,3616.40,8434.39,8597.38,4518.25,9344.37,9468.91,5608.05,12433.39,4520.00,0.00
4,12001907,102.04,0.00,153.06,0.00,1533.67,0.00,1020.41,0.00,90462.02,...,0.00,27845.75,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [27]:
# pivot table to show spending by agency by month
pivot_by_agency_by_month = pd.pivot_table(df_merged,
                              index='agency_name',
                              columns='yearmonth',
                              values='amt',
                              aggfunc=sum).reset_index().fillna(0.0)

In [28]:
pivot_by_agency_by_month.head()

yearmonth,agency_name,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211
0,AGRICULTURE & NAT. RESOURCES,4553984.51,5459280.11,3184810.95,4563026.45,4018080.91,4542845.11,2580667.52,3179380.41,1956272.74,...,3247797.68,2067205.97,1852042.80,3102042.79,3326132.93,5171343.90,5891628.91,8949588.70,10563589.24,11763195.79
1,ATTORNEY GENERAL,957625.66,735497.69,623860.35,785921.39,506460.41,993938.01,458631.43,852224.42,747638.70,...,334898.61,956193.81,357752.46,305413.46,898613.11,529646.34,911783.38,913003.27,498653.99,804492.73
2,BOARD OF REGENTS,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1028.97,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,BUREAU OF ADMINISTRATION,3289216.74,2438932.15,3071094.16,3312497.95,4100399.72,4503432.72,2236538.56,3323797.37,2926331.57,...,4226897.45,3700937.70,2560138.71,3117759.36,5497078.72,1474817.65,6005284.77,4081062.02,3402500.86,4074252.95
4,BUREAU OF FINANCE & MANAGEMENT,293867.53,29844357.42,16697248.80,45557236.23,26539516.40,272537295.00,236495927.62,64731642.55,10823657.69,...,2148851.95,2758240.29,2744899.91,2817788.82,1079694.89,1926380.91,783750.94,34499084.06,202489.70,158890.14


In [29]:
# sort descending by the latest month, which is the last column in the df
pivot_by_agency_by_month.sort_values(pivot_by_agency_by_month.columns[-1], ascending=False).head()

yearmonth,agency_name,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211
8,EDUCATION,65974788.11,65500549.66,116304643.26,68332112.21,70058588.75,78929033.24,73749212.31,68055283.43,69631573.80,...,89493695.56,83546910.35,87508057.26,88568985.40,80299969.12,74268676.59,102256778.79,98877600.50,79914322.60,94997855.29
29,TRANSPORTATION,112242554.86,80742028.73,85169178.59,88238239.08,44241706.16,48685747.67,24691890.25,17549055.92,55915847.53,...,20015968.78,22187009.63,60150307.36,59262619.25,73672893.84,79174566.51,98267822.05,99749651.25,90178433.71,67253740.11
22,REVENUE,80743795.95,61222012.16,38239412.77,61372287.64,36028745.61,39436444.01,98670075.57,32730046.06,36924114.16,...,34420900.05,45136545.54,60539801.80,65388669.96,49027433.62,84674601.63,63576109.00,56408581.13,72268477.01,45440456.88
25,SOCIAL SERVICES,8908296.62,14693605.58,9757778.16,15346485.23,11012697.79,11802287.82,14462991.12,6754884.10,11853614.22,...,42091811.10,16858631.56,13626664.62,14866358.69,25433346.91,44701410.47,23431482.68,14028707.11,16622820.26,14178498.50
0,AGRICULTURE & NAT. RESOURCES,4553984.51,5459280.11,3184810.95,4563026.45,4018080.91,4542845.11,2580667.52,3179380.41,1956272.74,...,3247797.68,2067205.97,1852042.80,3102042.79,3326132.93,5171343.90,5891628.91,8949588.70,10563589.24,11763195.79


### Filter to examine spending by one agency

Let's filter to look at spending by the state Department of Education.

In [30]:
# find the code to filter on
df_codes[df_codes['agency_name'].str.contains('education', case=False)]

,agency_code,agency_name
15,12,EDUCATION


In [31]:
education = df_merged[df_merged['agency'] == '12']

In [32]:
education.head()

,agency,document_date,document_number,vendor_name,vendor_number,vendor_group_number,ap_payment_date,voucher_number,amt,agency_code,agency_name,yearmonth
52423,12,2021-09-30,SC12320C-247-15,MTW SOLUTIONS LLC,12491439,NaN,2022-07-29,019713,30708.49,12,EDUCATION,202207
52424,12,2021-10-27,SC123221G-834-F,GRANT COUNTY SHERIFF,12054848,NaN,2022-07-15,005324,3050.00,12,EDUCATION,202207
52425,12,2021-12-06,32134,NOVEYA INCORPORATED,12053327,NaN,2022-08-03,020244,77.00,12,EDUCATION,202208
52426,12,2022-01-06,32322,NOVEYA INCORPORATED,12053327,NaN,2022-08-24,035897,77.00,12,EDUCATION,202208
52427,12,2022-02-11,32576,NOVEYA INCORPORATED,12053327,NaN,2022-08-31,041790,154.00,12,EDUCATION,202208


In [25]:
len(education)

41206

### Add up spending before and after a given date

Here, let's compare spending before and after July 1, 2021.

In [33]:
target_date = datetime(2021, 7, 1)

In [34]:
spending_before_date = df_merged[df_merged['ap_payment_date'] < target_date]
spending_after_date = df_merged[df_merged['ap_payment_date'] >= target_date]

In [35]:
print(f'Before: ${spending_before_date["amt"].sum():,.2f}')
print(f'After: ${spending_after_date["amt"].sum():,.2f}')

Before: $3,748,962,630.93
After: $4,927,989,330.10


### Break down payment frequency to vendors by month

Use case: See if payments went up or down, or stopped or started, to a vendor or a group of vendors at a given month. E.g., a new rule went into effect and you want to see if/how that affected the frequency/amount of spending, but instead of just a before/after sum you want a monthly breakdown of previous spending to get a sense of frequency.

In [49]:
# what's the target date of the change we're looking at?
target_date = datetime(2022, 10, 1)

# get it in a form that matches column names: yyyymm
# c.f. http://strftime.org
target_yearmonth = target_date.strftime('%Y%m')

In [50]:
# goal is to get a list of column names with yearmonth values
# before and after the target_yearmonth

# gonna do this one step at a time

# show column names
pivot_by_vendor_by_month.columns

Index(['vendor_number', '202007', '202008', '202009', '202010', '202011',
       '202012', '202101', '202102', '202103', '202104', '202105', '202106',
       '202107', '202108', '202109', '202110', '202111', '202112', '202201',
       '202202', '202203', '202204', '202205', '202206', '202207', '202208',
       '202209', '202210', '202211'],
      dtype='object', name='yearmonth')

In [51]:
# show column names except the initial vendor_number column
pivot_by_vendor_by_month.columns[1:]

Index(['202007', '202008', '202009', '202010', '202011', '202012', '202101',
       '202102', '202103', '202104', '202105', '202106', '202107', '202108',
       '202109', '202110', '202111', '202112', '202201', '202202', '202203',
       '202204', '202205', '202206', '202207', '202208', '202209', '202210',
       '202211'],
      dtype='object', name='yearmonth')

In [54]:
# figure out the index position of the target_yearmonth in the list of columns
# see list.index() docs https://docs.python.org/3/tutorial/datastructures.html
# n.b., you have to coerce the .columns object to a list() before you can use the .index() method,

target_col_idx = list(pivot_by_vendor_by_month.columns).index(target_yearmonth)

In [58]:
# doublecheck that the column name you're selecting for matches `target_yearmonth`
assert(pivot_by_vendor_by_month.columns[target_col_idx] == target_yearmonth)

In [64]:
# finally, get a list of columns with yearmonth values before ... (minus initial vendor_number col)
pre_date_columns = pivot_by_vendor_by_month.columns[1:target_col_idx]

# ... and after
post_date_columns = pivot_by_vendor_by_month.columns[target_col_idx:]

In [65]:
# next, goal is to add up total spending prior to this month and after
# define a function that will sum spending across a row for the
# selected columns

def sum_totals_yearmonth(row, direction='pre'):
    cols = pre_date_columns
    
    if direction == 'post':
        cols = post_date_columns

    return round(sum([row[x] for x in cols]), 2)

In [66]:
pivot_by_vendor_by_month['pre_date_total'] = pivot_by_vendor_by_month.apply(sum_totals_yearmonth, direction='pre', axis=1)
pivot_by_vendor_by_month['post_date_total'] = pivot_by_vendor_by_month.apply(sum_totals_yearmonth, direction='post',axis=1)

In [67]:
pivot_by_vendor_by_month.head()

yearmonth,vendor_number,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202204,202205,202206,202207,202208,202209,202210,202211,pre_date_total,post_date_total
0,12001820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,9218.52,1390.18,0.00,10608.70
1,12001823,53353.87,52533.01,52723.84,52346.98,52186.30,52215.97,51974.41,52402.83,51845.38,...,53015.49,52589.16,51873.09,51578.06,50749.81,51047.36,51306.75,52057.66,1398656.23,103364.41
2,12001827,0.00,0.00,0.00,0.00,0.00,0.00,675.00,0.00,736.00,...,0.00,0.00,372.00,0.00,0.00,0.00,0.00,0.00,1783.00,0.00
3,12001831,3029.28,10061.58,11085.74,4028.23,0.00,8776.72,6357.01,3394.56,5616.32,...,8597.38,4518.25,9344.37,9468.91,5608.05,12433.39,4520.00,0.00,184974.46,4520.00
4,12001907,102.04,0.00,153.06,0.00,1533.67,0.00,1020.41,0.00,90462.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,612680.57,0.00


In [78]:
# make a quick df to look up vendor names by number
vendor_lookup = df_merged[['vendor_number', 'vendor_name']].drop_duplicates(subset=['vendor_number'])

In [79]:
vendor_lookup.head()

,vendor_number,vendor_name
0,12403860,GRANT COUNTY REVIEW INC
1,12620107,BLACK HILLS LODGING LLC
2,STATE,SD HERITAGE STORE
3,12550348,INNOVATIVE OFFICE SOLUTIONS
5,12279233,AT&T MOBILITY II LLC


In [80]:
# merge with pivoted df
pivoted_with_vendor_names = pd.merge(
    pivot_by_vendor_by_month,
    vendor_lookup,
    how='left',
    on='vendor_number'
)

In [81]:
pivoted_with_vendor_names.head()

,vendor_number,202007,202008,202009,202010,202011,202012,202101,202102,202103,...,202205,202206,202207,202208,202209,202210,202211,pre_date_total,post_date_total,vendor_name
0,12001820,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,9218.52,1390.18,0.00,10608.70,LL BEAN INC
1,12001823,53353.87,52533.01,52723.84,52346.98,52186.30,52215.97,51974.41,52402.83,51845.38,...,52589.16,51873.09,51578.06,50749.81,51047.36,51306.75,52057.66,1398656.23,103364.41,UNUM LIFE INS CO OF AMERICA
2,12001827,0.00,0.00,0.00,0.00,0.00,0.00,675.00,0.00,736.00,...,0.00,372.00,0.00,0.00,0.00,0.00,0.00,1783.00,0.00,NATIONAL RURAL HEALTH ASSOC
3,12001831,3029.28,10061.58,11085.74,4028.23,0.00,8776.72,6357.01,3394.56,5616.32,...,4518.25,9344.37,9468.91,5608.05,12433.39,4520.00,0.00,184974.46,4520.00,IDEXX LABORATORIES INC
4,12001907,102.04,0.00,153.06,0.00,1533.67,0.00,1020.41,0.00,90462.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,612680.57,0.00,BAETE FORSETH HVAC


In [82]:
# make sure you ended up with the same number of records
assert(len(pivot_by_vendor_by_month) == len(pivoted_with_vendor_names))

In [83]:
# filter to get vendors who were paid something in the months before
# the date of interest but not after
some_before_none_after = pivoted_with_vendor_names[(pivoted_with_vendor_names['pre_date_total'] > 0) & (pivoted_with_vendor_names['post_date_total'] == 0)]

print(f'{len(some_before_none_after):,} vendors were paid something before the target month but nothing after')
print()

# starting with the month immediately preceding the target yearmonth,
# count up how many vendors were paid that month -- the
# goal is to get a sense of consecutive payments each month that stopped,
# and it's accomplished by gradually filtering the same dataframe
# with a filter that iterates backward by month

# loop over the list of pre-date columns in reverse
for i, yearmonth in enumerate(reversed(pre_date_columns), 1):

    month_phrase = f'{i} consecutive months'
    
    if i == 1:
        month_phrase = f'the month'
        

    # filter to get vendors who were paid more than 0 in this month
    some_before_none_after = some_before_none_after[some_before_none_after[yearmonth] > 0]
    
    # get the total number in this cohort
    total = len(some_before_none_after)
        
    plural_phrase = 'vendors were'
    
    if total == 1:
        plural_phrase = 'vendor was'

    msg = f'{total:>4,} {plural_phrase} paid something in the {month_phrase} before the target month, but nothing after'
    print(msg)

19,318 vendors were paid something before the target month but nothing after

1,129 vendors were paid something in the the month before the target month, but nothing after
 246 vendors were paid something in the 2 consecutive months before the target month, but nothing after
  89 vendors were paid something in the 3 consecutive months before the target month, but nothing after
  63 vendors were paid something in the 4 consecutive months before the target month, but nothing after
  38 vendors were paid something in the 5 consecutive months before the target month, but nothing after
  30 vendors were paid something in the 6 consecutive months before the target month, but nothing after
  23 vendors were paid something in the 7 consecutive months before the target month, but nothing after
  15 vendors were paid something in the 8 consecutive months before the target month, but nothing after
  12 vendors were paid something in the 9 consecutive months before the target month, but nothing af